#Notebook for creating mounts in Azure Data Lake Storage Gen2
####This notebook can be extrapolated for creation of mounts towards AWS (S3), changing the topics oriented to its authentication

###Initialization and configuration of variables for authentication to the blob storage, in this case ADSL Gen2
#####We will carry out the activities for the ADSL Gen2, using an Azure AD Main Service; and its id and secret we will have it stored in Azure Key Vault

https://docs.microsoft.com/en-us/azure/active-directory/develop/howto-create-service-principal-portal

#####If you don't have Key Vault, you could directly configure the Service Principal's id and secret using the scope secrets backed Databricks

https://www.mssqltips.com/sqlservertip/5914/securely-manage-secrets-in-azure-databricks-using-databricksbacked/

In [0]:
#Name the scope secret configured towards Key Vault
scope_name = "scope-secret-name"
#Name of the id of the service principal to use
sp_id_kv = "id-service-principal"
#Name of the service principal secret to use
sp_secret_kv = "secret-service-principal"
#Id tenant azure
id_tenant = "zzzz76cc-0ggf-4ww0-xxxx-ffff55a8yyyy"

#We obtain the value of the secrets stored in Key Vault
sp_id = dbutils.secrets.get(scope=scope_name,key=sp_id_kv)
sp_secret = dbutils.secrets.get(scope=scope_name,key=sp_secret_kv)

###Initialization of variables oriented to the Storage Account
#####We will base our exercise on a Datalake with the zoning proposed by Databricks to create a LakeHouse, as we see in the following image ...

![Delta Lake w/ Dimensional Schema Production](https://databricks-knowledge-repo-images.s3.us-east-2.amazonaws.com/DeltaLake/deltalake-dimensional-modeling.png)

In [0]:
#Name of the storage account on which we will create the containers
st_name = "stname001"

#List initialization for containers for mounts creation
containers = []
containers = ["bronze-layer", "silver-layer", "gold-layer"]

###Configuration for connection to the storage account using the Services Principal

In [0]:
config = {"fs.azure.account.auth.type": "OAuth",  
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": sp_id,
       "fs.azure.account.oauth2.client.secret": sp_secret,
       "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{id_tenant}/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "false"}

###Function definition for mount creation

In [0]:
def create_mounts (containers, st_name, config):
  for x in containers:
    mount_point = f"/mnt/{x}"
    
    try:
      if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        print(mount_point + " ==> Mount already exists")
      else:
        dbutils.fs.mount(
        source = f"abfss://{x}@{st_name}.dfs.core.windows.net",
        mount_point = mount_point, 
        extra_configs = config)
        print(mount_point + " ==> Mount created")
    except Exception as e:
       print(e)

###Executing the function

In [0]:
create_mounts(containers, st_name, config)

###Validate existing and created mounts

In [0]:
%fs ls dbfs:/mnt

###Disassemble units, if you have the need

In [0]:
#dbutils.fs.unmount("/mnt/consumption-zone")